#### **Feature importance um zu belegen, dass die Historie eines Users auschlaggebend für die Tringekeldwahrscheinlichkeit seiner aktullen Bestellung ist**

In [1]:
import pandas as pd
import plotly.express as px
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, recall_score, precision_score

In [2]:
def print_evaluation(pipeline_or_model: str, X_train, X_test, y_train, y_test, y_train_pred, y_test_pred):
    accurary_train = accuracy_score(y_train, y_train_pred)
    recall_train = recall_score(y_train, y_train_pred)
    precision_train = precision_score(y_train, y_train_pred)

    accurary_test = accuracy_score(y_test, y_test_pred)
    recall_test = recall_score(y_test, y_test_pred)
    precision_test = precision_score(y_test, y_test_pred)
    
    print(
        f"{pipeline_or_model} Evaluation:\n"
        f"{'':6} {'ACC':>10} | {'REC':>14} | {'PRE':>10} | {'rows':>8} | {'columns':>8}\n"
        f"{'Train':6} {accurary_train:10.5f} | {recall_train:14.5f} | {precision_train:10.5f} | {X_train.shape[0]:8} | {X_train.shape[1]:8}\n"
        f"{'Test':6} {accurary_test:10.5f} | {recall_test:14.5f} | {precision_test:10.5f} | {X_test.shape[0]:8} | {X_test.shape[1]:8}\n"
    )

In [3]:
df = pd.read_parquet("DSCB310 - UE2 - Shopping Carts.parquet")

In [4]:
df = df.reset_index(drop= True)
df = df.fillna(0)

In [5]:
df_userhh = df.copy()
df_userhh = df_userhh.sort_values(by= ["user_id", "order_number"], ascending= True)
df_userh1 = df_userhh.groupby("user_id", as_index= False).agg(orders = ("order_id", "unique"), reordered= ("reordered", "mean"), tip= ("tip", "mean"), order_number= ("order_number", "unique"))

last_orders = []
for ind, order in enumerate(df_userh1.orders):
 last_orders.append(df_userh1.orders[ind][-1])

df_userhh.drop(df_userhh.loc[df_userhh.order_id.isin(last_orders)].index, inplace= True)
df_userh = df_userhh.groupby("user_id", as_index= False).agg(orders = ("order_id", "nunique"), reordered= ("reordered", "mean"), tip= ("tip", "mean"))
df_last_order = df.loc[df.order_id.isin(last_orders)].groupby("user_id").agg(tip_last_order= ("tip", "max"), last_order_id = ("order_id", "max")).reset_index()
#df_userh = df_userh.merge(df_last_order, how= "right", on= "user_id")

In [6]:
df1 = df.copy()

In [7]:
df_last_orders = df1.loc[df1.order_id.isin(last_orders)]

In [8]:
df1.drop(df1.loc[df1.order_id.isin(last_orders)].index, inplace= True)

In [9]:
df_tip = df1.groupby(["user_id"], as_index= False).agg(user_tip_ratio = ("tip", "mean"))

In [10]:
df1 = df1.merge(df_tip, on= "user_id", how= "right")

In [17]:
df_last_orders = df_last_orders.merge(df_tip, on= "user_id", how= "right")

In [18]:
X = df1[["order_id",	"product_id",	"add_to_cart_order",	"user_id",	"order_number",	"order_dow",	"order_hour_of_day",	"days_since_prior_order",	"aisle_id",	"department_id", "user_tip_ratio"]]
y = df1["tip"]

In [19]:
X_train, y_train = X , y

In [20]:
X_test, y_test = df_last_orders[["order_id",	"product_id",	"add_to_cart_order",	"user_id",	"order_number",	"order_dow",	"order_hour_of_day",	"days_since_prior_order",	"aisle_id",	"department_id", "user_tip_ratio"]], df_last_orders["tip"]

In [14]:
gs_crossv = GridSearchCV(
 estimator= DecisionTreeClassifier(),
 param_grid= dict(min_samples_split= [2, 10, 20, 50, 100], max_depth= [1, 3, 6, 9], random_state= [0]),
 scoring = "accuracy",
 n_jobs= -1,
 cv= 5
).fit(X_train, y_train)

print(f"Best parameters: {gs_crossv.best_params_}, with score: {gs_crossv.best_score_}")

Best parameters: {'max_depth': 6, 'min_samples_split': 2, 'random_state': 0}, with score: 0.7724332017517882


In [21]:
dtc2 = DecisionTreeClassifier(min_samples_split= 2, max_depth= 6, random_state= 0).fit(X_train, y_train)

y_pred_train_best_dtc2 = dtc2.predict(X_train)
y_pred_test_best_dtc2 = dtc2.predict(X_test)

print_evaluation("DTC", X_train, X_test, y_train, y_test, y_pred_train_best_dtc2, y_pred_test_best_dtc2)

DTC Evaluation:
              ACC |            REC |        PRE |     rows |  columns
Train     0.77942 |        0.73144 |    0.77023 |  5740420 |       11
Test      0.76509 |        0.64272 |    0.69115 |   392249 |       11



In [22]:
print(dict(zip(X.columns, dtc2.feature_importances_)))

{'order_id': 6.701943163374691e-05, 'product_id': 0.0, 'add_to_cart_order': 6.541085648993934e-06, 'user_id': 0.0, 'order_number': 0.0, 'order_dow': 0.04376369120058702, 'order_hour_of_day': 0.018309682286576402, 'days_since_prior_order': 0.0, 'aisle_id': 0.0, 'department_id': 0.0, 'user_tip_ratio': 0.9378530659955538}


- feature_importance von user_tip_ratio mit 0.9379 am höchsten
- damit ist die Tip-History eines users auschschlag gebend für die Tip-wahrscheinlichkeit bei der aktuellen Bestellung 